<a href="https://colab.research.google.com/github/jc533/Song_data_game/blob/attackofthetitan-patch-1/data_game_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import

In [406]:
import sys
!cp ../input/rapids/rapids.0.14.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.10/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.10"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/
import warnings
warnings.filterwarnings('ignore')

'cp' is not recognized as an internal or external command,
operable program or batch file.
The system cannot find the path specified.
'cp' is not recognized as an internal or external command,
operable program or batch file.


In [433]:
import numba
numba.set_num_threads(20)

In [407]:
from sklearn.preprocessing import MultiLabelBinarizer

In [408]:
import pandas as pd
import numpy as np
import os
# from google.colab import drive

!pip install missingno
import missingno as msno

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
# drive.mount('/content/drive')
!pip3 install lightgbm
import lightgbm as lgbm

# data viz
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
import plotly.express as px
%matplotlib inline

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


# read file

In [409]:
path = 'C:/Users/ethan/downloads/datagame-2023'
train = pd.read_parquet(f'{path}/label_train_source.parquet')
test = pd.read_parquet(f'{path}/label_test_source.parquet')
target = pd.read_parquet(f'{path}/label_train_target.parquet')
composer = pd.read_parquet(f'{path}/meta_song_composer.parquet')
genre = pd.read_parquet(f'{path}/meta_song_genre.parquet')
lyricist = pd.read_parquet(f'{path}/meta_song_lyricist.parquet')
producer = pd.read_parquet(f'{path}/meta_song_producer.parquet')
titletext = pd.read_parquet(f'{path}/meta_song_titletext.parquet')
song = pd.read_parquet(f'{path}/meta_song.parquet')


In [411]:
tmp = pd.DataFrame()
tmp["song_id"] = song["song_id"]
label_encoder.fit(tmp["song_id"])
# mapping["song_id"] = song["song_id"]
unmapping = dict(zip(label_encoder.transform(label_encoder.classes_,),label_encoder.classes_))
mapping = dict(zip(label_encoder.classes_,label_encoder.transform(label_encoder.classes_,)))

In [412]:
mlb = MultiLabelBinarizer(classes=genre['genre_id'].unique())
genre_df = pd.DataFrame(mlb.fit_transform(genre['genre_id']),columns=genre['genre_id'].unique())
genre_df = pd.concat([genre['song_id'], genre_df], axis=1)
genre_df = genre_df.groupby('song_id').sum().reset_index()
genre_df

,song_id,b856b6781d370a3645c6dde0c20b3597,2eabe9f164346c7b3ff1bd23078f483e,bb3d7b04b67d5aeb5ab145bdd70750da,ce4db56f6a48426643b08038139a8a75,43244ec4c30a0dad837d982892bc0c05,bb737ea7450c3abbab1ff613a2c6309e,5f2a134d2289a8a3de6663ee8e248c8a,6ea61b86b8fff0e3a05bc73ea4eaf21f,1d7c8bb87dcc1457ed90240c06f9ebdf,...,1f4b914a79eb2bb01c2ea694af626625,03c358e326d99a3863e044c5d8e9fb50,d36204de09c0c6084d55b1f484a23773,f6f02545c1905a8c72c5bf006579996e,ee1696df08c0ac6005e3b9442cdbbded,1a619fcf3adfd91711699b7e2cf2c367,80117354556efbf237f0020bea2f7e42,c3ad773a264597f9f46c2d666e1a8b50,74694a488312db91fcd56818fed8b3a6,5455e1699025b025a3523aba4719e818
0,00002d2ddb6c789bcb8ef14f52339e39,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,000049836b2e92a6a361e40d7f0c73db,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,00005d588ff5b96c84556f17b716e227,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,00005ee00af3e6bc97c0a694bae95728,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,000067e6edb45097956ee81bbd1edb63,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706791,ffffb17f494477ddef244df20d662e01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
706792,ffffc0ee02f3fac2b4a10ed7a6c3423f,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
706793,ffffcf62a064a729b55618a2648c6af0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
706794,ffffe1b0071c87b3716f708f7066a22f,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# merge data


In [413]:
#song_data = pd.concat([train, target])
#song_data = song_data.sort_values(by=['session_id', 'listening_order'])
#song_data = pd.merge(train, target, on='session_id', how='inner')
song_data = pd.merge(train, song, on='song_id', how='left')
#song_data = pd.merge(song_data, composer, on='song_id', how='left')
song_data = pd.merge(song_data, genre_df, on='song_id', how='left')
#song_data = pd.merge(song_data, lyricist, on='song_id', how='left')
#song_data = pd.merge(song_data, producer, on='song_id', how='left')
#song_data = pd.merge(song_data, titletext, on='song_id', how='left')
song_vali = pd.merge(target, song, on='song_id', how='left')
#song_vali = pd.merge(song_vali, composer, on='song_id', how='left')
song_vali = pd.merge(song_vali, genre_df, on='song_id', how='left')
#song_vali = pd.merge(song_vali, lyricist, on='song_id', how='left')
#song_vali = pd.merge(song_vali, producer, on='song_id', how='left')
#song_vali = pd.merge(song_vali, titletext, on='song_id', how='left')


In [414]:
print(train.shape)
print(song_data.shape)

(11445180, 6)
(11445180, 49)


In [415]:
composer['song_id'].is_unique

False

In [416]:
song_test = pd.merge(test, song, on='song_id', how='left')
#song_test = pd.merge(song_test, composer, on='song_id', how='left', validate='m:1')
song_test = pd.merge(song_test, genre_df, on='song_id', how='left')
#song_test = pd.merge(song_test, lyricist, on='song_id', how='left')
#song_test = pd.merge(song_test, producer, on='song_id', how='left')
#song_test = pd.merge(song_test, titletext, on='song_id', how='left')
print(test.shape)
print(song_test.shape)

(2861280, 6)
(2861280, 49)


In [417]:
song_data.drop_duplicates(subset=['session_id', 'song_id', 'listening_order'], inplace=True)
song_vali.drop_duplicates(subset=['session_id', 'song_id', 'listening_order'], inplace=True)

In [418]:
for i in song_data.select_dtypes(include=['object']).columns:
  song_data[i][song_data[i].isnull()] = 'unknown'
song_data = song_data.fillna(value=0)

for i in song_vali.select_dtypes(include=['object']).columns:
  song_vali[i][song_vali[i].isnull()] = 'unknown'
song_vali = song_vali.fillna(value=0)


for i in song_test.select_dtypes(include=['object']).columns:
  song_test[i][song_test[i].isnull()] = 'unknown'
song_test = song_test.fillna(value=0)


encol = [ x for x in song_data.columns if x not in ["song_id"] ]
for i in encol:
  if i!="song_id":
    song_data[i] = label_encoder.fit_transform(song_data[i])
encol = [ x for x in song_vali.columns if x not in ["song_id"] ]
for i in encol:
  song_vali[i] = label_encoder.fit_transform(song_vali[i])
encol = [ x for x in song_test.columns if x not in ["song_id"] ]
for i in encol:
  song_test[i] = label_encoder.fit_transform(song_test[i])

song_data["song_id"] = song_data["song_id"].map(mapping)
song_vali["song_id"] = song_vali["song_id"].map(mapping)
song_test["song_id"] = song_test["song_id"].map(mapping)

song_data.reset_index(drop=True)
song_vali.reset_index(drop=True)
song_test.reset_index(drop=True)

,session_id,song_id,unix_played_at,play_status,login_type,listening_order,artist_id,song_length,album_id,language_id,...,1f4b914a79eb2bb01c2ea694af626625,03c358e326d99a3863e044c5d8e9fb50,d36204de09c0c6084d55b1f484a23773,f6f02545c1905a8c72c5bf006579996e,ee1696df08c0ac6005e3b9442cdbbded,1a619fcf3adfd91711699b7e2cf2c367,80117354556efbf237f0020bea2f7e42,c3ad773a264597f9f46c2d666e1a8b50,74694a488312db91fcd56818fed8b3a6,5455e1699025b025a3523aba4719e818
0,118,271639,299096,2,2,0,5333,218,6876,12,...,0,0,0,0,0,0,0,0,0,0
1,118,589665,299203,2,2,1,4126,233,4308,12,...,0,0,0,0,0,0,0,0,0,0
2,118,331055,299237,2,2,2,165,230,5000,8,...,0,0,0,0,0,0,0,0,0,0
3,118,297816,299238,2,2,3,1693,215,13707,12,...,0,0,0,0,0,0,0,0,0,0
4,118,759365,299240,2,2,4,4379,192,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2861275,142898,80547,2038413,1,2,15,48613,82,118126,8,...,0,0,0,0,0,0,0,0,0,0
2861276,142898,659924,2038435,1,2,16,48613,102,118126,8,...,0,0,0,0,0,0,0,0,0,0
2861277,142898,626741,2038468,1,2,17,48613,97,118126,8,...,0,0,0,0,0,0,0,0,0,0
2861278,142898,909527,2038488,1,2,18,48613,91,118126,8,...,0,0,0,0,0,0,0,0,0,0


In [419]:
song_data.shape[0]/20
song_vali.shape[0]/5
song_test.shape[0]/20

143064.0

In [420]:
training = song_data.groupby('session_id')
targeting = song_vali.groupby('session_id')
testing = song_test.groupby('session_id')
#training = song_data.merge()

# Preprocess


In [421]:
%%time
t = {}
# t = {i: training.get_group(i).to_numpy() for i in training.groups}
# df_t = pd.DataFrame(t)
k=0
s=100000
#s=1
size = len(song_data)/20
itm = training.groups.items()
for sess_id, group_indices in itm:
    tmp = song_data.loc[group_indices]
    tmp.drop(columns = ['session_id'],inplace=True)
    tmp = tmp.to_numpy().flatten()
    # print(tmp)
    tmp = tmp.tolist()
#     print(type(song_data.loc[group_indices].to_nump))
    t[sess_id] = tmp
    if k%s==0 and k>=s:
      #print(tmp)
      #break
      print(k,k/size*100)
    k+=1
#df_t = pd.DataFrame(t)

100000 17.474605030239804
200000 34.94921006047961
300000 52.423815090719415
400000 69.89842012095922
500000 87.37302515119903
CPU times: total: 14min 33s
Wall time: 15min 31s


In [422]:
df_t = pd.DataFrame.from_dict(t)
df_t

,0,1,2,3,4,5,6,7,8,9,...,572249,572250,572251,572252,572253,572254,572255,572256,572257,572258
0,994225,14405,437836,44719,949607,9941,634844,329315,60289,918698,...,289521,385131,563546,760641,601974,892975,211986,253613,685466,940213
1,589474,1544057,3409065,683315,803698,916502,5571094,5186785,119360,119957,...,4680943,1166781,635563,3763173,1185777,4013424,53341,4972100,4079619,3928165
2,1,2,2,2,1,1,2,1,2,2,...,7,3,1,1,1,4,1,1,3,2
3,2,2,2,13,2,2,2,2,2,2,...,2,3,3,2,3,3,2,3,2,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
956,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
957,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
958,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [423]:
song_test.shape
#2739710, 2739712, 2739714, 2739715, 2739716, 2739719, 2739721, 2739723, 2739727, 2739728, 2739729, 2739732, 2739734

(2861280, 49)

In [424]:
%%time
te = {}
# t = {i: training.get_group(i).to_numpy() for i in training.groups}
# df_t = pd.DataFrame(t)
k=0
s=100000
#s=1
size = len(song_test)/20
itm = testing.groups.items()
for sess_id, group_indices in itm:
    tmp = song_test.loc[group_indices]
    tmp.drop(columns = ['session_id'],inplace=True)
    tmp = tmp.to_numpy().flatten()
    # print(tmp)
    tmp = tmp.tolist()
#     print(type(song_data.loc[group_indices].to_nump))
    te[sess_id] = tmp
    if k%s==0 and k>=s:
      #print(tmp)
      #break
      print(k,k/size*100)
    k+=1
df_te = pd.DataFrame(te)
df_te = df_te.transpose()

100000 69.89878655706538
CPU times: total: 3min 52s
Wall time: 4min 2s


In [434]:
%%time
v = {}
# t = {i: training.get_group(i).to_numpy() for i in training.groups}
# df_t = pd.DataFrame(t)
k=0
s=100000
# s=1
size = len(song_vali)/5
itm = targeting.groups.items()


for sess_id, group_indices in itm:
    tmp = song_vali.loc[group_indices, 'song_id'].to_numpy()
    # tmp = np.take(tmp, 1)
    tmp = tmp.tolist()

#     print(type(song_data.loc[group_indices].to_nump))
    v[sess_id] = tmp

    if k%s==0 and k>=s:
      # break
      # print(tmp)
      print(k,k/size*100)
    k+=1
#df_v = pd.DataFrame(v)
#df_v

100000 17.474605030239804
200000 34.94921006047961
300000 52.423815090719415
400000 69.89842012095922
500000 87.37302515119903
CPU times: total: 1min 29s
Wall time: 1min 36s


# Train

In [429]:
df_v = pd.DataFrame.from_dict(v)
df_v

,0,1,2,3,4,5,6,7,8,9,...,572249,572250,572251,572252,572253,572254,572255,572256,572257,572258
0,714573,827206,649931,993467,987875,101458,481537,879764,321975,92839,...,53839,776460,563546,675403,502061,243958,727401,154284,785711,865600
1,66954,523535,541297,912199,617147,222834,86305,167308,183542,745216,...,367695,901098,563546,760641,478645,647039,890711,409034,685466,313485
2,4081,954908,792701,993467,987875,910490,667533,879764,559588,842457,...,327633,264931,563546,817777,898521,374691,786880,680838,842585,408659
3,140308,103310,317012,912199,617147,727800,973667,958286,146397,538151,...,692564,369117,563546,212330,386071,239341,985016,690473,973141,42347
4,862634,360635,867247,993467,723216,204125,27481,167308,918698,412091,...,569619,219772,563546,694460,990338,340380,985016,7274,842585,365513


In [430]:
y


,0,1,2,3,4
0,243821,22841,1399,47751,294176
1,282056,178483,326023,35198,122868
2,221784,184548,270421,107890,295765
3,338932,311287,338932,311287,338932
4,337032,210603,337032,210603,246733
...,...,...,...,...,...
572254,83055,220797,127606,81482,115814
572255,248101,303815,268458,336063,336063
572256,52410,139295,232259,235589,2449
572257,268031,233871,287276,332116,287276


In [431]:
X = df_t.transpose() #= pd.DataFrame.from_dict(t)
y = df_v.transpose()

#y.shape

In [435]:
#X = training.drop(columns=['song_id'])
#y = training.song_id
X_train, X_val, Y_train, Y_val = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [436]:
params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': 'gbdt',
        'learning_rate': 0.3 ,
        'verbose': 0,
        'num_leaves': 108,
        'bagging_fraction': 0.95,
        'bagging_freq': 1,
        'bagging_seed': 1,
        'feature_fraction': 0.9,
        'feature_fraction_seed': 1,
        'max_bin': 256,
        'max_depth': 10,
        'num_rounds': 200,
        'metric' : 'auc'
    }
train_data = lgbm.Dataset(X_train, Y_train)
train_vali = lgbm.Dataset(X_val, Y_val)
#model_f1 = lgbm.train(params, train_set=, )

lgbmodel = lgbm.LGBMRegressor(learning_rate=0.05, n_estimators=100)
model = MultiOutputRegressor(lgbmodel)

model.fit(X_train, Y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.157551 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29883
[LightGBM] [Info] Number of data points in the train set: 429194, number of used features: 180
[LightGBM] [Info] Start training from score 515909.350685
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.137340 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29883
[LightGBM] [Info] Number of data points in the train set: 429194, number of used features: 180
[LightGBM] [Info] Start training from score 516234.883386
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.136380 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29883
[LightGBM] [Info] Number of data points in the train set: 429194, number of used features: 180
[LightGB

MultiOutputRegressor(estimator=LGBMRegressor(learning_rate=0.05))

In [507]:
y_pred = model.predict(df_te)
y_pred.shape

(143064, 5)

In [508]:
y_pred = pd.DataFrame(y_pred).astype(int)
#print(y_pred.shape)
for i in y_pred.columns:
  y_pred[i] = y_pred[i].map(unmapping)

In [509]:
session_test = test['session_id'].drop_duplicates().reset_index().drop(['index'], axis=1)
#print(session_test.shape)

y_pred = pd.concat([session_test, y_pred], axis=1)
y_pred

(143064, 1)


,session_id,0,1,2,3,4
0,598,7fcd95d1ca46ba3262065573c23ffb51,83e5d61b87bf5f1725774da29141bd6b,7f49a09c91c580889cc7f21e589b2146,81c22af1a9089f3d952c5fc7ced2e9b8,7f824ecedffca0b2ad2bc72bc1a621fb
1,1039,80e669ac5c2191aaa4620488e270e216,8214f699b16c48eb223be15beb4cfda2,80d4289b8db5d8433b2bf6d96a44ec9a,80041f2881c7f4759c1fab6fddbd66bc,802237adbe5c5672cf2c53e6110fbd47
2,1199,7dd0e3fc5dc33679740328f772744193,7cb239946fe71cf4309c95df3b235c41,7d641bb8471dda49452f34c82a42bbef,7c7fd48c757752a71b2267118d1113b0,7c3ce7a85964109bc882218aa6b054a2
3,1489,7f1d1dcc1cd71e241f9188f14080fce5,80cc3ca62ad78b3489e052015b142ee5,7e30082b3e675c417f6800abd24ac9eb,7e1b2f4a1b42206468f9b9e636b66650,7f16f44b56a69125d0d8fb2b797114e9
4,1868,7ec7e1490a1a4baff88cab02ade3fa08,7e54fbbea15c4f06aafc012c4072b0eb,80b83fa0f4af7a82de5bb09c59dd67bc,80220ea5b7558b877d76e383c1072bda,8163ec991bfb83f9d0113ba9cd99c33d
...,...,...,...,...,...,...
143059,621109,80602a3774b4d7fa89db779a03cc32ff,82f4146e4a8f3235965d103b826ce99e,7f6fb9f86f608c48c232884f11079ae8,819c95215fffa4822aa52bb263ea09ae,807c50306805141a54a24d9e90456907
143060,621339,7dd2820c3d176d1541cfecc26a83df03,7e18a5daa87e989da71a55197be77e38,7dc7ab1e8d1e2e835db46fba9e0e5c91,7b98e2541e6158835584faae86d07827,7ca7f02793c61318894065e4f3307c51
143061,712209,7eff07017d251eb82c47e7d642f454b1,84dc5f3e9e1cb6773a40ede68c76686f,7c615ec5d7d772d6716074e158fd366d,81b017cb8e966c8efe1e594a4b776e15,80755bbcb3ed31fbead8abd7c5f43f9a
143062,713779,835d039c33327080d7139dd2cd523676,82ebe0d360a1b0792bd1f0633af3ef2f,81edbbcc4049691c90ea7873cd0663a6,82f4028b31e96ed2f7460847cbb57f41,8138ce80d898700d6ed9dd820b77f48f


In [481]:
y_pred.shape

(2861280, 6)

In [510]:
trainingcsv = df_t.to_csv('./training.csv')
targetingcsv = df_v.to_csv('./targeting.csv')
testingcsv = df_te.to_csv('./testing.csv')

KeyboardInterrupt: ignored

In [445]:
testingcsv = df_te.to_csv('./testing.csv')

In [512]:
y_pred.to_csv('./sublgbm.csv', header=['session_id', 'top1', 'top2', 'top3', 'top4', 'top5'], index=False)